In [16]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
load_dotenv()

True

In [2]:
def get_file_type(file_path):
  if file_path.endswith((".c", ".h")):
    return "code"
  elif file_path.endswith(".rst"):
    return "documentation"
  elif file_path.endswith("Makefile"):
    return "makefile"
  else:
    return None
    raise ValueError(f"Unknown file type: {file_path}")

In [3]:
def iterate_over_files(project_source):
    for element in project_source:
        for root, dirs, files in os.walk(element):
            for file in files:
                yield os.path.join(root, file)

In [4]:
persist_directory = "./storage"
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
embeddings = OpenAIEmbeddings()
project_source = "rag\\Linux\\"

d:\miniforge3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
import threading

def process_directory_concurrently(directory_path):
    content_lock = threading.Lock()  # Lock for thread-safe content concatenation
    content = ""

    def concat_file_content(file_path):
        nonlocal content  # Access the outer 'content' variable
        if get_file_type(file_path) is not None:
            with open(file_path, "r", encoding="iso-8859-1") as f:
                with content_lock:  # Acquire lock for thread-safe access
                    content += "//" + file_path + "\n"
                    content += f.read()

    threads = []
    for root, _, files in os.walk(directory_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            thread = threading.Thread(target=concat_file_content, args=(file_path,))
            threads.append(thread)
            thread.start()

    # WAIT FOR ALL THREADS TO FINISH BEFORE RETURNING
    for thread in threads:
        thread.join()

    return content


In [6]:
# concatenated_contents = ""
# for file in iterate_over_files(project_source):
#     if get_file_type(file) is not None:
#         with open(file, "r", encoding="iso-8859-1") as f:
#             concatenated_contents += "//" + file + "\n"
#             tmp = f.read()
#             concatenated_contents += tmp

In [7]:
content = process_directory_concurrently(project_source)

In [17]:
docs = text_splitter.split_text(content)
db = Chroma.from_texts(texts=docs,
                    embedding=embeddings,
                    persist_directory=persist_directory)
print("Content loaded into ChromaDB!")

Created a chunk of size 1806, which is longer than the specified 1024
Created a chunk of size 1163, which is longer than the specified 1024
Created a chunk of size 1411, which is longer than the specified 1024
Created a chunk of size 1125, which is longer than the specified 1024
Created a chunk of size 1181, which is longer than the specified 1024
Created a chunk of size 1809, which is longer than the specified 1024
Created a chunk of size 10400, which is longer than the specified 1024
Created a chunk of size 1733, which is longer than the specified 1024
Created a chunk of size 1326, which is longer than the specified 1024
Created a chunk of size 1181, which is longer than the specified 1024
Created a chunk of size 1031, which is longer than the specified 1024
Created a chunk of size 1417, which is longer than the specified 1024
Created a chunk of size 1159, which is longer than the specified 1024
Created a chunk of size 1189, which is longer than the specified 1024
Created a chunk of 

Created a chunk of size 1082, which is longer than the specified 1024
Created a chunk of size 1930, which is longer than the specified 1024
Created a chunk of size 1089, which is longer than the specified 1024
Created a chunk of size 1069, which is longer than the specified 1024
Created a chunk of size 1267, which is longer than the specified 1024
Created a chunk of size 1360, which is longer than the specified 1024
Created a chunk of size 1123, which is longer than the specified 1024
Created a chunk of size 1095, which is longer than the specified 1024
Created a chunk of size 1248, which is longer than the specified 1024
Created a chunk of size 1340, which is longer than the specified 1024
Created a chunk of size 3813, which is longer than the specified 1024
Created a chunk of size 3693, which is longer than the specified 1024
Created a chunk of size 1486, which is longer than the specified 1024
Created a chunk of size 2636, which is longer than the specified 1024
Created a chunk of s

Content loaded into ChromaDB!


d:\miniforge3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [24]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
# https://github.com/langchain-ai/langchain/issues/1532

In [21]:
prompt_file = "prompts\KernelGuru.txt"
PROMPT = open(prompt_file).read()
task = open("prompts/task.txt").read()
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [22]:
rag_chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever)

task = "Here are the bug details, please help resolve the issue and provide appropriate fix with code.\n" + task
response = rag_chain.invoke(task)

In [23]:
response

{'query': 'Here are the bug details, please help resolve the issue and provide appropriate fix with code.\nPlease help to resolve issue in the linux kernel.\nBug 218514\nSummary:\tiwlwifi null pointer dereference when debugfs=off\nProduct:\tDrivers\tReporter:\tdrastic_endurance425\nComponent:\tnetwork-wireless-intel\tAssignee:\tDefault virtual assignee for network-wireless-intel (drivers_network-wireless-intel)\nStatus:\tNEW ---\t \t \nSeverity:\tnormal\tCC:\tdrastic_endurance425\nPriority:\tP3\t \t \nHardware:\tAll\t \t \nOS:\tLinux\t \t \nKernel Version:\t\nRegression:\tNo\nAttachments:\tjournalctl log\nDescription: drastic_endurance425 2024-02-20 14:47:42 UTC\nCreated attachment 305900 [details]\njournalctl log\nSome commit part of the 6.7 kernel series has resulted in the bootup sequence to issue me an oops. One of my kernel parameters is debugfs=off and NetworkManager exits with a kernel NULL pointer dereference. This issue is present whether I use the vanilla kernel 6.7.5 or the 